<h1> <center>  Méthodes de Valorisation dans le Modèle de Black-Scholes pour les Options Européennes </center> </h1>

[Lien vers le Modèle de Black-Scholes](https://en.wikipedia.org/wiki/Black%E2%80%93Scholes_model). 



## Table
   - [Parité Call-Put](#sec1)
   - [Méthode de Monte-Carlo](#sec2)
   - [Arbre Binominal](#sec3)


In [59]:
import numpy as np
from scipy.stats import norm,sem

In [106]:
class BS_Pricer():
    def __init__(self,S,K,T,r,sigma):
        self.S=S
        self.K=K
        self.T=T
        self.r=r
        self.sigma=sigma
        
        
    def BSPrice(self,payoff):
           
        d1 = (np.log(self.S/self.K) + (self.r + self.sigma**2 / 2) * self.T) / (self.sigma * np.sqrt(self.T))
        d2 = (np.log(self.S/self.K) + (self.r - self.sigma**2 / 2) * self.T) / (self.sigma * np.sqrt(self.T))

        if payoff=="call":
            return self.S * norm.cdf( d1 ) - self.K * np.exp(-self.r * self.T) *norm.cdf( d2 )
        elif payoff=="put":
            return self.K * np.exp(-self.r * self.T) * norm.cdf( -d2 ) - self.S * norm.cdf( -d1 )
        else:
            raise ValueError("Type Invalide: Choississez 'call' ou 'put'")
            
    def MonteCarloPrice(self,payoff,N):
        W = norm.rvs( (self.r-0.5*self.sigma**2)*self.T , np.sqrt(self.T)*self.sigma, N)
        S_T = self.S * np.exp(W)
        if payoff=="call":
            call = np.mean( np.exp(-r*T) * np.maximum(S_T-self.K,0) )
            call_err = sem( np.exp(-r*T) * np.maximum(S_T-self.K,0) )
            return call,call_err # standard error
        if payoff=="put":
            put = np.mean( np.exp(-r*T) * np.maximum(self.K-S_T,0))
            put_err = sem( np.exp(-r*T) * np.maximum(self.K-S_T,0) )  
            return put,put_err# standard error
        
    def BinomialPrice(self,payoff,N):
        dT = float(self.T) / N # Delta t
        u = np.exp(self.sigma * np.sqrt(dT))                 # up factor
        d = 1.0 / u                                   # down factor 
        V = np.zeros(N+1)                             # initialize the price vector
        S_T = np.array( [(self.S * u**j * d**(N - j)) for j in range(N + 1)] )  # price S_T at time T
        a = np.exp(self.r * dT)    # risk free compounded return
        p = (a - d)/ (u - d)  # risk neutral up probability
        q = 1.0 - p           # risk neutral down probability   
        if payoff =="call":
            V[:] = np.maximum(S_T-K, 0.0)
        elif payoff=="put":
            V[:] = np.maximum(K-S_T, 0.0)
        else:
            raise ValueError("Type Invalide: Choississez 'call' ou 'put'")            
        for i in range(N-1, -1, -1):
            V[:-1] = np.exp(-r*dT) * (p * V[1:] + q * V[:-1])    # the price vector is overwritten at each step
        return V[0]
        

        

In [107]:
S=100
K=100
T=1
r=0.05
sigma=0.2


In [108]:
Option = BS_Pricer(S,K,T,r,sigma)

print("Prix du Call : ", Option.BSPrice("call") )  
print("Prix du Put: ", Option.BSPrice("put") ) 

Prix du Call :  10.450583572185565
Prix du Put:  5.573526022256971


<a id='sec1'></a>
### Parité Call-Put

La formule de la Parité Call-Put qui se démontre en AOA est visible ici :  [wiki page](https://en.wikipedia.org/wiki/Put%E2%80%93call_parity)

$$ Call - Put = S_0 - K e^{-rT}  $$


In [119]:
print(Option.BSPrice("call")) 
print(Option.BSPrice("put") + S - K * np.exp(-r*T) )

10.450583572185565
10.450583572185565


<a id='sec2'></a>
## Méthode de Monte-Carlo

Nous allons simuler N variables

$$ S_T^i = S_0 e^{(r -\frac{1}{2}\sigma^2)T + \sigma W_{T}^i} $$

pour $1 \leq i \leq N$.    
On utilise alors l'approximation Monte-Carlo pour le calcul:

$$ \mathbb{E}^{\mathbb{Q}}\biggl[ (S_T - K)^+ \bigg| S_0 \biggr] \; 
\approx \; \frac{1}{N} \sum_{i=1}^N (S_T^i - K)^+
$$

Pour une option de Put, On utilise le payoff $(K - S_T )^+$ dans l'espérance.

In [118]:
N=100000
CallMonteCarlo,CallStandardError=Option.MonteCarloPrice("call",N)
PutMonteCarlo,PutStandardError=Option.MonteCarloPrice("put",N)
print("Prix du Call: {}, avec erreur: {}".format(CallMonteCarlo, CallStandardError))
print("Prix du Put: {}, avec erreur: {}".format(PutMonteCarlo, PutStandardError))

Prix du Call: 10.444463003001719, avec erreur: 0.04655049804469559
Prix du Put: 5.580142091615531, avec erreur: 0.027446950291453963


<a id='sec3'></a>
## Arbre Binomial

On s'intéresse dorénavant à l'implémentation de la méthode par arbre binomial recombinant

In [116]:
N=1000
CallBinominalPrice=Option.BinomialPrice("call",N)
PutBinominalPrice=Option.BinomialPrice("put",N)

print("Prix du call : {}".format(CallBinominalPrice))
print("Prix du Put : {}".format(PutBinominalPrice))

Prix du call : 10.448584103764572
Prix du Put : 5.57152655383368
